In [1]:
import numpy as np
import numpy.ma as ma
import geopandas as gpd
import fiona
import timing

# auxiliary libraries
import subprocess
from subprocess import Popen, PIPE
import warnings
import yaml
import os
from osgeo import ogr, osr, gdal

# for appending scripts and functions
import sys

# local modules
import text_matching
from vector_proc import VectorTransform
from reprojection import RasterTransform  # this imports RasterTransform class

class RasterMetadata():
    def __init__(self, x_min: str, x_max: str, y_min: str, y_max: str, cell_size: str, xres: str, yres: str, is_cartesian: str, crs_info:str) -> None:
        self.x_min = x_min
        self.x_max = x_max
        self.y_min = y_min
        self.y_max = y_max
        self.cell_size = cell_size
        self.xres = xres
        self.yres = yres
        self.is_cartesian = is_cartesian
        self.crs_info = crs_info
        
#TODO for now put all general functions here
class config_loader():
    def __init__(self, config_path:str) -> None:
        self.config = self.load_yaml(config_path)

    def load_yaml(self, path:str) -> dict:
        """
        Load a yaml file from the given path to a dictionary

        Args:
            path (str): path to the yaml file

        Returns:
            dict: dictionary containing the yaml file content
        """
        with open(path , 'r') as file:
            return yaml.safe_load(file)

#TODO rename this    
class utils():
    def __init__(self) -> None:
        pass

    def extract_layer_names(aelf, gpkg_path:str) -> list:
        """
        Extracts layer names from a GeoPackage file.

        Args:
            gpkg_path (str): path to the GeoPackage file
        
        Returns:
            list: list of layer names in the GeoPackage file
        """
        with fiona.Env():
            layer_names = fiona.listlayers(gpkg_path)
        return layer_names
    
    
    def check_raster_projection(self, lulc:str):
        """
        Check the projection of the input raster dataset.

        Args:
            lulc (str): path to the input raster dataset

        Returns:
            Raster_Properites: object containing raster metadata
        """
        rt = RasterTransform(lulc)
        
        xres, yres = rt.check_res()
        x_min, x_max, y_min, y_max, cell_size = rt.get_raster_info()

        # print the results
        print(f"x_min: {x_min}")
        print(f"x_max: {x_max}")
        print(f"y_min: {y_min}")
        print(f"y_max: {y_max}")
        print(f"Spatial resolution of input raster dataset (cell size): {cell_size}")

        # check if the input raster dataset has a projected (cartesian) CRS
        is_cartesian, crs_info = rt.check_cart_crs()

        # cast to Raster_Properites object
        return RasterMetadata(x_min, x_max, y_min, y_max, cell_size, xres, yres, is_cartesian, crs_info)
    
    def merge_tiffs_into_vrt(self, tiffs:list, output_path:str):
        """
        Merge multiple raster datasets into a single VRT file.

        Args:
            tiffs (list): list of paths to the raster datasets
            output_path (str): path to the output VRT file

        Returns:
            None
        """
        # write the list to a new file (path to the file is ../data/list_of_tiff_files.txt)
        tiffs_filepaths = output_path.replace('.vrt', '_tiffs.txt')
        with open(tiffs_filepaths, "w") as f:
            for item in tiffs:
                f.write(item + "\n")

        gdal_command = f"""gdalbuildvrt -input_file_list {tiffs_filepaths} {output_path}"""
        proc = Popen(gdal_command, shell=True, stdout=PIPE, stderr=PIPE)
        stdout, stderr = proc.communicate()
        # remove the list of tiff files
        os.remove(tiffs_filepaths)

        if proc.returncode != 0:
            print(proc.returncode)
            print("STDERR:", stderr.decode())
            raise Exception("Error creating VRT")
        
       

    def extract_attribute_values(self, vector_gpkg:str, layer_name:str=None, attribute:str="highway") -> list:
        """
        Extract all unique attribute values from a vector GeoPackage file.
        
        Args:
            vector_gpkg (str): path to the vector GeoPackage file
            attribute (str): attribute name to extract values from
            
        Returns:
        list: list of unique attribute values
        """

        # open the vector data source
        data_source = ogr.Open(vector_gpkg)
        if data_source is None:
            raise RuntimeError(f"Failed to open the vector file: {vector_gpkg}")

        # get the layer from the data source (use first if not specified)
        if layer_name is None:
            layer_name = data_source.GetLayer(0).GetName()
            print(f"Layer name not specified. Using the first layer: {layer_name}")
        layer = data_source.GetLayerByName(layer_name)
        print(f"Layer name: {layer_name}")

        # get unique values of the attribute
        values = layer.GetNextFeature()
        unique_values = set()
        while values:
            value = values.GetField(attribute)
            unique_values.add(value)
            values = layer.GetNextFeature()

        return unique_values
        


# PROCESSORS

class vector_data_preprocessor(utils):
    def __init__(self, config: dict, parent_dir:str, vector_dir:str, year:int, lulc_crs:int, lulc_is_cartesian:bool) -> None:
        super().__init__()
        self.config = config
        self.year = year
        self.crs = lulc_crs
        self.is_cartesian = lulc_is_cartesian
        self.vector_refine = os.path.normpath(os.path.join(parent_dir,vector_dir,self.load_auxillary_data()))
        print(f"Path to the input vector dataset: {self.vector_refine}")
        self.vector_layer_names = self.check_vector_data(self.vector_refine, self.crs)
        # specify the output directory
        self.vector_railways_buffered = os.path.join(parent_dir,vector_dir, f"railways_{self.year}_buffered.gpkg")
        self.vector_roads_buffered = os.path.join(parent_dir,vector_dir, f"roads_{self.year}_buffered.gpkg")

        # TODO UNCOMMENT buffer the features
        self.buffer_features('railways', self.vector_railways_buffered)
        self.buffer_features('roads', self.vector_roads_buffered)
        # raise Exception("STOPPED HERE")
    

    def load_auxillary_data(self):
        # specify input vector data
        osm_data_template = self.config.get('osm_data', None)
        vector_refine = None # define a new variable which will be equal either osm_data or user_vector (depending on the configuration file)
        if osm_data_template is not None:
            osm_data = osm_data_template.format(year=self.year)
            user_vector = None
            vector_refine = osm_data 
            print ("Input raster dataset will be enriched with OSM data.")
        else:
            warnings.warn("OSM data not found in the configuration file.") 
            user_vector_template = self.config.get('user_vector',None)
            if user_vector_template is not None:
                user_vector = user_vector_template.format(year=self.year)
                vector_refine = user_vector
                print ("Input raster dataset will be enriched with user-specified data.")
            else:
                raise ValueError("No valid input vector data found. Neither OSM data nor user specified data found in the configuration file.")
            
        # print the name of chosen vector file
        print(f"Using vector file to refine raster data: {vector_refine}")
        return vector_refine
    

    def check_vector_data(self, vector_refine:str, crs:int):
        vector_layer_names = self.extract_layer_names(vector_refine) 
        print(f"Layers found in the input vector file: {vector_layer_names}")
        formatted_layers = ', '.join(vector_layer_names)  # join layer names with a comma and space for readability
        print(f"Please continue if the layers in the vector file listed below are correct \n: {formatted_layers}.")

        #   TODO UNCOMMENT check vector data
        # define full path with vector input directory
        # split path on last occurence of '/' and take the first part
        filepath = os.sep.join(vector_refine.split(os.sep)[:-1])
        vector_refine_path = os.path.join(filepath)

        # check if crs matches input raster (lulc). If not, reproject the vector data
        Vt = VectorTransform(vector_refine_path)
        files_to_validate = Vt.reproject_vector(crs, overwrite=True)
        if len(files_to_validate) > 0:
            Vt.fix_geometries_in_gpkg(Vt.geom_valid(files_to_validate), overwrite=True)
        return vector_layer_names
    
    def buffer_features(self, layer:str, output_filepath:str, epsg:int=27700):
        """
        Buffer the features in the input vector layer. 
        If the instance is not in cartesian coordinates, a temporary transformation is used to apply the buffer in meters and then transform back to the original CRS.
        """
        if os.path.exists(output_filepath):
            os.remove(output_filepath)

        subquery = f"""
            CASE 
                WHEN "width" IS NULL OR CAST("width" AS REAL) IS NULL THEN 
                    CASE 
                        WHEN highway IN ('motorway', 'motorway_link', 'trunk', 'trunk_link') THEN 30/2
                        WHEN highway IN ('primary', 'primary_link', 'secondary', 'secondary_link') THEN 20/2 
                        ELSE 10/2 
                    END 
                ELSE CAST("width" AS REAL)/2 
            END
        """
        # if it is not in cartesian coordinates, transform the geometry to a temporary cartesian CRS for buffering and then back to the original CRS
        if self.is_cartesian == False:
            query = f"""
                ST_Transform(
                    ST_Buffer(
                        ST_Transform(geom, {epsg}),
                        {subquery}
                    ),
                    {self.crs}
                ) AS geometry,
                *
            """
        else:
            query = f""" ST_Buffer(geom, {subquery}) AS geometry, * """


        print(f"Buffering {layer} layer...")
        #NOTE only for roads and railways for now
        ogr2ogr_command = [
            'ogr2ogr',
            '-f', 'GPKG',
            output_filepath, # output file path
            self.vector_refine, # input file path (should be before the SQL statement)
            '-dialect', 'SQLite',
            '-sql', f"""
                SELECT
                {query}
                FROM {layer}; /* to specify layer of input file */
            """,
            '-nln', layer, # define layer in the output file
            '-nlt', 'POLYGON' # ensure the output is a polygon
        ]

        # execute ogr2ogr command
        try:
            result = subprocess.run(ogr2ogr_command, check=True, capture_output=True, text=True)
            print(f"Successfully buffered {layer} layer and saved to {output_filepath}.")
            if result.stderr:
                print(f"Warnings or errors:\n{result.stderr}")
        except subprocess.CalledProcessError as e:
            print(f"Error buffering roads: {e.stderr}")
        except Exception as e:
            print(f"Unexpected error: {str(e)}")
    

class lulc_data_preprocessor(utils):
    def __init__(self, config:dict, lulc_dir:str, parent_dir:str) -> None:
        super().__init__()
        self.config = config
        print(f"Path to the input raster dataset: {lulc_dir}")

        impedance_file = self.config.get('impedance', None)
        impedance_dir = self.config.get('impedance_dir', None)

        if impedance_file is not None and impedance_dir is not None:
            # define path
            impedance_file = os.path.join(parent_dir,impedance_dir,impedance_file)
            print(f"Using auxiliary tabular data from {impedance_file}.")
        else:
            warnings.warn("auxiliary tabular data was not provided.")

        # map LULC codes to OSM features 
        self.lulc_codes = self.lulc_mapping(impedance_file)
        self.raster_metadata = self.check_raster_projection(lulc_dir)
        

    def lulc_mapping(self, impedance_file:str):
        """
        Map LULC codes to OSM features using either user-defined mapping or text-matching tool with the impedance file.

        Args:
            impedance_file (str): path to the impedance file
        
        Returns:
            dict: dictionary containing LULC codes and corresponding OSM features
        """
        # find out from config file if user wants define LULC codes on their own, or use text-matching tool
        user_matching = self.config.get('user_matching')
      
        # if user defines mapping on their own
        if user_matching.lower() == 'true': # case-insensitive condition
            # access variables and subvariables from the confiration file
            lulc_codes = self.config.get('lulc_codes', {})
            # print codes of areas from OSM corresponding with LULC codes from input raster dataset
            print("User-specified mapping of LULC codes and OSM features is used.")
            print("LULC dictionary:", lulc_codes)

        # if user defines mapping from text-matching tool
        elif user_matching.lower() == 'false': # case-insensitive condition
            # call the function and capture the result
            lulc_codes = text_matching.codes(self.config, impedance_file)
            # print codes of areas from OSM corresponding with LULC codes from input raster dataset
            print("Text matching tool used to map LULC codes and corresponding OSM features.")
            print("LULC dictionary:", lulc_codes)
        else:
            raise ValueError("User did not specify mapping between OSM features and LULC types.")
        
        return lulc_codes

class Auxillary_data_processor(config_loader, utils):
    
    def __init__(self, config:dict, parent_dir:str, lulc:str, year:int) -> None:
        self.config = config
        self.year = year
        self.parent_dir = parent_dir
        self.vector_dir = self.config.get('vector_dir')
        self.output_dir = self.config.get('output_dir')
        self.lulc = lulc

        ## LULC PREPROCESSING
        self.lp = lulc_data_preprocessor(self.config, self.lulc, self.parent_dir)
        
        ## OSM PREPROCESSING
        self.vp = vector_data_preprocessor(self.config, self.parent_dir, self.vector_dir, self.year,self.lp.raster_metadata.crs_info["epsg"], self.lp.raster_metadata.is_cartesian)

        ## rasterize vector layers
        self.rasters_temp = self.rasterize_vector_layers()

        ## PART 2 MERGING RASTER LAYERS
        lulc_upd = os.path.normpath(os.path.join(self.parent_dir,self.output_dir,f'lulc_{year}_upd.tif'))

        # TODO REMOVE to print the output filename
        print(f"Enriched land-use/land-cover dataset(s) will be fetched to {lulc_upd}")
        #TODO REMOVE debug: print dimensions for each raster to check them against LULC dimension
        self.check_raster_dimensions([self.lulc, *self.rasters_temp])

        # overwrite rasters over input dataset in the following order: waterbodies, waterways, roads, railways
        output_data, output_ds, nodata_value = self.overwrite_raster(self.lulc, *self.rasters_temp)
        self.write_raster(output_data, output_ds, lulc_upd, nodata_value)

        ## PART 3 IMPEDANCE RECALCULATION
        # TODO add this after refactoring everything else
    
    # @DeprecationWarning
    def check_raster_dimensions(self, listraster_uri:list): 
        for raster_path in listraster_uri:
            dataset = gdal.Open(raster_path)
            if dataset:
                width = dataset.RasterXSize
                height = dataset.RasterYSize
            else:
                raise ValueError(f"Unable to open raster file: {raster_path}")
            print(f"Dimensions of {os.path.basename(raster_path)}: {width} x {height}")


    def write_raster(self, output_data:any, output_ds:any, output_raster:str, nodata_value:int):
        """
        Write a new raster dataset from the given data array.

        Args:
            output_data (np.array): data array to write to the raster
            output_ds (gdal.Dataset): dataset of the input raster
            output_raster (str): path to the output raster dataset
            nodata_value (int): no data value for the output raster

        Returns:
            None
        """

        # get the driver to write a new GeoTIFF
        driver = gdal.GetDriverByName('GTiff')
        out_ds = driver.Create(output_raster, output_ds.RasterXSize, output_ds.RasterYSize, 1, gdal.GDT_Byte)

        # set geo-transform and projection from the input raster
        out_ds.SetGeoTransform(output_ds.GetGeoTransform())
        out_ds.SetProjection(output_ds.GetProjection())

        # write the data to the output raster
        out_band = out_ds.GetRasterBand(1)
        out_band.WriteArray(output_data)

        # set nodata value 
        out_band.SetNoDataValue(nodata_value)

        # flush the data and close files
        out_band.FlushCache()
        out_ds = None  # close the file
        output_ds = None  # close the input file

        print(f"Output raster saved to {output_raster}")

    def new_layer_from_attributes(self, vector_gpkg:str, layer_name:str, attribute:str, value:str, output_gpkg:str):
        """
        Create a new layer from the input layer based on the attribute value.

        Args:
            vector_gpkg (str): path to the input vector GeoPackage file
            layer_name (str): name of the layer to extract features from
            attribute (str): attribute name to filter by
            value (str): value to filter by
            output_gpkg (str): path to the output GeoPackage file

        """
        print("Layer to to access:", layer_name)
        ogr_command = f"""
            ogr2ogr -f GPKG {output_gpkg} {vector_gpkg} -sql "SELECT * FROM {layer_name} WHERE {attribute} LIKE '%{value}%'"
        """
        print(ogr_command)
        proc = Popen(ogr_command, shell=True, stdout=PIPE, stderr=PIPE)
        stdout, stderr = proc.communicate()
        if proc.returncode != 0:
            raise RuntimeError(stderr)
        print(f"New layer saved to {output_gpkg}")

        # # define ogr2ogr command
        # ogr2ogr_cmd = [
        #     'ogr2ogr',
        #     '-f', 'GPKG',
        #     output_gpkg,
        #     vector_gpkg,
        #     '-dialect', 'SQLite',
        #     '-sql', sql_statement
        # ]

        # # execute ogr2ogr command through subprocess
        # subprocess.run(ogr2ogr_cmd, check=True)
        
        return output_gpkg
   
    def rasterize_vector_roads(self, raster_metadata:str , roads_gpkg:str, burn_value:int, groupby_roads:bool=True):
        """
        Rasterize roads vector layer to be used for enriching the LULC dataset.

        Args:
            raster_metadata (RasterMetadata): object containing raster metadata (extent, cell size, etc.)
            roads_gpkg (str): path to the roads GeoPackage file
            burn_val (int): value to burn into the output raster 
        Returns:
            str: path to the rasterized layer
        """

        #extract road types from roads geopackage

        #TODO just hard code the layer name instead since we know it's roads?
        # road_layer_name = [layer for layer in self.vp.vector_layer_names if 'road' in layer.lower()][0]
        road_layer_name = 'roads'
        road_types = self.extract_attribute_values(roads_gpkg, road_layer_name, attribute='highway')
        print(f"Road types found in the input vector file: {road_types}")

        #group attributes by first suffix (e.g. primary, secondary, tertiary) split by '_'
        if groupby_roads:
            road_types = list(set([road_type.split('_')[0] for road_type in road_types]))
            print(f"Road types to be rasterized: {road_types}")
        
        # for each road type, rasterize the roads
        road_tiffs = []
        for road_type in road_types:
            # create a new layer for each road type
            output_path = os.path.join(self.parent_dir,self.output_dir,f'roads_{road_type}.gpkg')
            road_gpkg = self.new_layer_from_attributes(roads_gpkg, road_layer_name, 'highway', road_type, output_path)
            # edit roads path to include road type
            output_path = output_path.replace('.gpkg', f'_{self.year}.tif')
            self.rasterize_vector_layer(raster_metadata, road_gpkg, output_path, nodata_value=0, burn_value=burn_value, layer_name=f'roads_{road_type}')
            # delete the temporary layer
            os.remove(road_gpkg)
            road_tiffs.append(output_path)

        # build a roads.vrt file to merge all road types
        self.merge_tiffs_into_vrt(road_tiffs, os.path.join(self.parent_dir,self.output_dir,f'roads_{self.year}.vrt'))

    def rasterize_vector_layers(self):
        """
        Rasterize OSM vector layers to be used for enriching the LULC dataset.

        Returns:
            list: list of paths to the rasterized layers
        """
        roads = os.path.join(self.parent_dir,self.output_dir,f'roads_{self.year}.vrt')
        railways = os.path.join(self.parent_dir,self.output_dir,f'railways_{self.year}.tif')
        waterbodies = os.path.join(self.parent_dir,self.output_dir,f'waterbodies_{self.year}.tif')
        waterways = os.path.join(self.parent_dir,self.output_dir,f'waterways_{self.year}.tif')
        rasters_temp = [waterbodies, waterways, roads, railways] # Order is important for next steps

        # rasterize roads and railways from buffered geometries
        # TODO UNCOMMENT
        self.rasterize_vector_roads(self.lp.raster_metadata, self.vp.vector_roads_buffered, burn_value=self.lp.lulc_codes["lulc_road"])
        self.rasterize_vector_layer(self.lp.raster_metadata,self.vp.vector_railways_buffered, railways, nodata_value=0, burn_value=self.lp.lulc_codes["lulc_railway"])

        # rasterize waterbodies and waterways from the initial input vector data
        self.rasterize_vector_layer(self.lp.raster_metadata,self.vp.vector_refine, waterbodies, layer_name='waterbodies', nodata_value=0, burn_value=self.lp.lulc_codes["lulc_water"]) # read from the corresponding layer
        self.rasterize_vector_layer(self.lp.raster_metadata,self.vp.vector_refine, waterways, layer_name='waterways', nodata_value=0, burn_value=self.lp.lulc_codes["lulc_water"]) # read from the corresponding layer

        return rasters_temp
    
    def rasterize_vector_layer(self,lulc:RasterMetadata, vector_path:str, output_path:str, nodata_value:str, burn_value:str, layer_name:str=None):
        """
        Rasterize a vector layer to a raster dataset.

        Args:
            lulc (Raster_Properites): object containing raster properties
            vector_path (str): path to the vector dataset
            output_path (str): path to the output raster dataset
            nodata_value (str): no data value for the output raster
            burn_value (str): value to burn into the output raster
            layer_name (str): name of the layer to rasterize (optional)

        Returns:
            str: path to the output raster dataset
        """
        # open the vector data source
        data_source = ogr.Open(vector_path)
        if data_source is None:
            raise RuntimeError(f"Failed to open the vector file: {vector_path}")

        # check the number of layers and write it to the variable
        layer_count = data_source.GetLayerCount()
        
        # define gdal_rasterize command
        #TODO get extent from lulc raster
        gdal_rasterize_cmd = [
            'gdal_rasterize',
            '-tr', str(lulc.cell_size), str(lulc.cell_size),  # output raster pixel size
            '-te', str(lulc.x_min), str(lulc.y_min), str(lulc.x_max), str(lulc.y_max),  # output extent 
            '-a_nodata', str(nodata_value),  # no_data value
            '-ot', 'Int16',   # output raster data type,
            '-burn', str(burn_value),  # burn-in value
            '-at',  # all touched pixels are burned in
            vector_path,  # input vector file
            output_path  # output raster file
        ]

         # add the layer name if there are multiple layers 
        if layer_count > 1: # specify layer name if using merged geopackage as an input file
            gdal_rasterize_cmd.insert(1, '-l')
            gdal_rasterize_cmd.insert(2, str(layer_name))

        # execute gdal_rasterize command through subprocess
        subprocess.run(gdal_rasterize_cmd, check=True, capture_output=True, text=True)

        # compress output 
        output_compressed = output_path.replace('.tif', '_compr.tif')
        gdal_translate_cmd = [
            'gdal_translate',
            output_path,
            output_compressed,
            '-co', 'COMPRESS=LZW',
            '-ot', 'Byte'
        ]
        # execute gdal_translate command through subprocess
        subprocess.run(gdal_translate_cmd, check=True)

        # rename compressed output to original
        os.remove(output_path)
        os.rename(output_compressed, output_path)

        print("Rasterized output saved to:", output_path)
        return output_path


    #function to overwrite values from input raster by multiple rasters
    def overwrite_raster(self, base_raster:str, *rasters:str):
        """
        Merge multiple rasters by overwriting values from the base raster with valid data from other rasters.

        Args:
            base_raster (str): path to the base raster dataset
            *rasters (str): paths to other raster datasets to be merged
        
        Returns:
            np.array: merged raster dataset
            gdal.Dataset: dataset of the base raster
            float: nodata value of the base raster
        """
        # open the input raster and read it
        base_ds = gdal.Open(base_raster)
        base_band = base_ds.GetRasterBand(1)
        base_data = base_band.ReadAsArray().astype(np.float32)
        
        # get nodata value for the input raster
        nodata_value = base_band.GetNoDataValue()
        if nodata_value is None:  # if nodata value is not defined, set 0 as a default
            nodata_value = 0
        base_data[base_data == nodata_value] = np.nan  # replace nodata value with nan for processing
        print(f"Nodata value of the input raster dataset: {nodata_value}")
        
        # iterate over other rasters
        for raster in rasters:
            ds = gdal.Open(raster)
            band = ds.GetRasterBand(1)
            data = band.ReadAsArray().astype(np.float32)
            current_nodata = band.GetNoDataValue()
            if current_nodata is None:  # handle missing nodata value
                current_nodata = 0
            data[data == current_nodata] = np.nan  # replace nodata with nan for processing
            
            # overwrite values in base_data where current raster has valid data
            mask = ~np.isnan(data)
            base_data[mask] = data[mask]
        
        # after processing, replace NaNs with the nodata value before saving
        base_data[np.isnan(base_data)] = nodata_value
        
        return base_data, base_ds, nodata_value
    
class preprocessor(config_loader):
    def __init__(self, config_path:str, output_dir:str) -> None:
        super().__init__(config_path)
        self.years = self.config.get('year', None)
        if self.years is None:
            warnings.warn("Year variable is null or not found in the configuration file... \n Defaulting to 2018")
            self.years = [2018]
        elif isinstance(self.years, int):
            self.years = [self.years]

        lulc_template = self.config.get('lulc', None)
        # substitute year from the configuration file
        year = self.years[0]
        lulc = lulc_template.format(year=year)

        print(f"Input raster to be used for processing is {lulc}.")

        parent_dir = os.getcwd()
        lulc_dir = self.config.get('lulc_dir')
        output_dir = self.config.get('output_dir')

        # create the output directory if it does not exist
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            print(f"Created directory: {output_dir}")
        
        # specifying the path to input files through the path variables
        lulc = os.path.normpath(os.path.join(parent_dir,lulc_dir,lulc))
        adp = Auxillary_data_processor(self.config, parent_dir,lulc,year)
        vector_refine = adp.vp.vector_refine

        print(f"Path to the input raster dataset: {lulc}")
        print(f"Path to the input vector dataset: {vector_refine}")

In [2]:
preprocessor('config.yaml', 'output')

Input raster to be used for processing is lulc_ukceh_25m_2018.tif.
Path to the input raster dataset: /data/data/input/lulc/lulc_ukceh_25m_2018.tif
Using auxiliary tabular data from /data/data/input/impedance/reclassification_ukceh.csv.
User-specified mapping of LULC codes and OSM features is used.
LULC dictionary: {'lulc_road': 20, 'lulc_railway': 21, 'lulc_water': 14, 'lulc_urban': None, 'lulc_suburban': None}
Good news! The spatial resolution of your raster data is consistent between X and Y.
Input raster dataset /data/data/input/lulc/lulc_ukceh_25m_2018.tif was opened successfully.
Coordinate reference system of the input raster dataset is EPSG:27700
x_min: 347225.0
x_max: 452300.0
y_min: 343800.0
y_max: 540325.0
Spatial resolution of input raster dataset (cell size): 25.0
Good news! The CRS of your input raster dataset is the Cartesian (projected) one.
Input raster dataset will be enriched with OSM data.
Using vector file to refine raster data: osm_merged_2018.gpkg
Path to the inpu

/opt/venv/lib/python3.12/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


CRS of the vector dataset 'osm_merged_2018.gpkg' is EPSG:27700.
CRS of the vector dataset 'osm_merged_2018.gpkg' matches the specified CRS: EPSG:27700.
Good news! All vector geometries in GeoPackage 'osm_merged_2018.gpkg' (layer 'railways') are valid.
----------------------------------------
Good news! All vector geometries in GeoPackage 'osm_merged_2018.gpkg' (layer 'roads') are valid.
----------------------------------------
Good news! All vector geometries in GeoPackage 'osm_merged_2018.gpkg' (layer 'waterbodies') are valid.
----------------------------------------
Good news! All vector geometries in GeoPackage 'osm_merged_2018.gpkg' (layer 'waterways') are valid.
----------------------------------------
Fixed geometries and saved to /data/data/input/vector/osm_merged_2018.gpkg.
Buffering railways layer...
Successfully buffered railways layer and saved to /data/data/input/vector/railways_2018_buffered.gpkg.
Warnings or errors:
Warning 1: A geometry of type LINESTRING is inserted int